In [1]:
import json
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import re
from datetime import datetime
from textstat import flesch_reading_ease
import numpy as np

In [2]:
class InvestmentDashboard:
    def __init__(self, json_data=None, json_file_path=None):
        """
        Initialize the dashboard with JSON data
        Args:
            json_data: Dictionary containing investment data
            json_file_path: Path to JSON file containing investment data
        """
        if json_file_path:
            with open(json_file_path, 'r') as f:
                self.data = json.load(f)
        elif json_data:
            self.data = json_data
        else:
            raise ValueError("Either json_data or json_file_path must be provided")
        
        self.analysis = self._analyze_data()
        
    def _analyze_data(self):
        """Analyze the JSON data to extract key metrics and insights"""
        analysis = {}
        
        for date, data in self.data.items():
            company = data.get('company_of_interest', 'Unknown')
            
            # Extract technical indicators from market report
            analysis[date] = {
                'company': company,
                'trade_date': data.get('trade_date'),
                'technical_indicators': self._extract_technical_indicators(data.get('market_report', '')),
                'sentiment_metrics': self._extract_sentiment_metrics(data.get('sentiment_report', '')),
                'fundamental_metrics': self._extract_fundamental_metrics(data.get('fundamentals_report', '')),
                'final_decisions': self._extract_decisions(data),
                'risk_assessment': self._analyze_risk_debate(data.get('risk_debate_state', {})),
                'news_sentiment': self._analyze_news_sentiment(data.get('news_report', ''))
            }
            
        return analysis
    
    def _extract_technical_indicators(self, market_report):
        """Extract technical indicators from market report"""
        indicators = {}
        
        # Price information
        price_match = re.search(r'closing price.*?\$?([\d,]+\.?\d*)', market_report, re.IGNORECASE)
        if price_match:
            indicators['closing_price'] = float(price_match.group(1).replace(',', ''))
        
        # Moving averages
        sma_50_match = re.search(r'50-SMA.*?\$?([\d,]+\.?\d*)', market_report, re.IGNORECASE)
        if sma_50_match:
            indicators['sma_50'] = float(sma_50_match.group(1).replace(',', ''))
            
        sma_200_match = re.search(r'200-SMA.*?\$?([\d,]+\.?\d*)', market_report, re.IGNORECASE)
        if sma_200_match:
            indicators['sma_200'] = float(sma_200_match.group(1).replace(',', ''))
            
        ema_10_match = re.search(r'10-EMA.*?\$?([\d,]+\.?\d*)', market_report, re.IGNORECASE)
        if ema_10_match:
            indicators['ema_10'] = float(ema_10_match.group(1).replace(',', ''))
        
        # MACD
        macd_match = re.search(r'MACD.*?([\d.]+)', market_report, re.IGNORECASE)
        if macd_match:
            indicators['macd'] = float(macd_match.group(1))
            
        # RSI
        rsi_match = re.search(r'RSI.*?([\d.]+)', market_report, re.IGNORECASE)
        if rsi_match:
            indicators['rsi'] = float(rsi_match.group(1))
            
        # ATR
        atr_match = re.search(r'ATR.*?([\d.]+)', market_report, re.IGNORECASE)
        if atr_match:
            indicators['atr'] = float(atr_match.group(1))
            
        return indicators
    
    def _extract_sentiment_metrics(self, sentiment_report):
        """Extract sentiment metrics from sentiment report"""
        metrics = {}
        
        # Reddit mentions
        mentions_match = re.search(r'(\d+)\s+mentions', sentiment_report, re.IGNORECASE)
        if mentions_match:
            metrics['reddit_mentions'] = int(mentions_match.group(1))
            
        # Sentiment score
        sentiment_match = re.search(r'sentiment score.*?(\d+\.?\d*)%?', sentiment_report, re.IGNORECASE)
        if sentiment_match:
            metrics['sentiment_score'] = float(sentiment_match.group(1))
            
        # Rule of 40 score
        rule40_match = re.search(r'Rule of 40.*?(\d+)%', sentiment_report, re.IGNORECASE)
        if rule40_match:
            metrics['rule_of_40'] = int(rule40_match.group(1))
            
        # Stock price from sentiment
        price_match = re.search(r'current price.*?\$?([\d,]+\.?\d*)', sentiment_report, re.IGNORECASE)
        if price_match:
            metrics['current_price'] = float(price_match.group(1).replace(',', ''))
            
        return metrics
    
    def _extract_fundamental_metrics(self, fundamentals_report):
        """Extract fundamental metrics from fundamentals report"""
        metrics = {}
        
        # Revenue
        revenue_match = re.search(r'revenue.*?\$?([\d,]+\.?\d*)\s*(billion|million)', fundamentals_report, re.IGNORECASE)
        if revenue_match:
            value = float(revenue_match.group(1).replace(',', ''))
            unit = revenue_match.group(2).lower()
            if unit == 'billion':
                value *= 1000
            metrics['revenue_millions'] = value
            
        # Revenue growth
        revenue_growth_match = re.search(r'(\d+)%.*?year-over-year', fundamentals_report, re.IGNORECASE)
        if revenue_growth_match:
            metrics['revenue_growth'] = int(revenue_growth_match.group(1))
            
        # Net income
        net_income_match = re.search(r'net income.*?\$?([\d,]+\.?\d*)\s*million', fundamentals_report, re.IGNORECASE)
        if net_income_match:
            metrics['net_income_millions'] = float(net_income_match.group(1).replace(',', ''))
            
        # Market cap
        market_cap_match = re.search(r'market capitalization.*?\$?([\d,]+\.?\d*)\s*(billion|trillion)', fundamentals_report, re.IGNORECASE)
        if market_cap_match:
            value = float(market_cap_match.group(1).replace(',', ''))
            unit = market_cap_match.group(2).lower()
            if unit == 'trillion':
                value *= 1000
            metrics['market_cap_billions'] = value
            
        return metrics
    
    def _extract_decisions(self, data):
        """Extract investment decisions from various reports"""
        decisions = {}
        
        # Extract final decisions from each report
        for report_type in ['market_report', 'sentiment_report', 'fundamentals_report']:
            report = data.get(report_type, '')
            decision_match = re.search(r'FINAL TRANSACTION PROPOSAL:\s*\*?\*?([A-Z]+)', report, re.IGNORECASE)
            if decision_match:
                decisions[report_type] = decision_match.group(1).upper()
                
        # Final trade decision
        final_decision = data.get('final_trade_decision', '')
        if 'BUY' in final_decision.upper():
            decisions['final_decision'] = 'BUY'
        elif 'SELL' in final_decision.upper():
            decisions['final_decision'] = 'SELL'
        elif 'HOLD' in final_decision.upper():
            decisions['final_decision'] = 'HOLD'
            
        return decisions
    
    def _analyze_risk_debate(self, risk_debate):
        """Analyze risk debate to extract risk assessment"""
        if not risk_debate:
            return {}
            
        risk_analysis = {}
        
        # Analyze judge decision
        judge_decision = risk_debate.get('judge_decision', '')
        if 'BUY' in judge_decision.upper():
            risk_analysis['risk_adjusted_decision'] = 'BUY'
        elif 'SELL' in judge_decision.upper():
            risk_analysis['risk_adjusted_decision'] = 'SELL'
        elif 'HOLD' in judge_decision.upper():
            risk_analysis['risk_adjusted_decision'] = 'HOLD'
            
        # Extract position sizing
        position_match = re.search(r'(\d+\.?\d*)%.*?portfolio', judge_decision, re.IGNORECASE)
        if position_match:
            risk_analysis['recommended_position_size'] = float(position_match.group(1))
            
        # Extract stop loss
        stop_loss_match = re.search(r'(\d+)%.*?stop', judge_decision, re.IGNORECASE)
        if stop_loss_match:
            risk_analysis['stop_loss_percent'] = int(stop_loss_match.group(1))
            
        return risk_analysis
    
    def _analyze_news_sentiment(self, news_report):
        """Analyze news report for sentiment indicators"""
        sentiment = {}
        
        # Count positive/negative words
        positive_words = ['growth', 'increase', 'strong', 'positive', 'bullish', 'surge', 'rise', 'improvement']
        negative_words = ['decline', 'fall', 'weak', 'negative', 'bearish', 'drop', 'concern', 'risk']
        
        news_lower = news_report.lower()
        positive_count = sum(news_lower.count(word) for word in positive_words)
        negative_count = sum(news_lower.count(word) for word in negative_words)
        
        if positive_count + negative_count > 0:
            sentiment['news_sentiment_score'] = (positive_count - negative_count) / (positive_count + negative_count) * 100
        else:
            sentiment['news_sentiment_score'] = 0
            
        return sentiment
    
    def create_dashboard(self):
        """Create the main dashboard with multiple visualizations"""
        fig = make_subplots(
            rows=4, cols=2,
            subplot_titles=(
                'Technical Indicators Overview', 'Investment Decisions Summary',
                'Sentiment Analysis', 'Fundamental Metrics',
                'Risk Assessment', 'Price vs Moving Averages',
                'Decision Consensus', 'Key Performance Metrics'
            ),
            specs=[[{"secondary_y": True}, {"type": "pie"}],
                   [{"type": "bar"}, {"type": "bar"}],
                   [{"type": "indicator"}, {"type": "scatter"}],
                   [{"type": "bar"}, {"type": "table"}]],
            vertical_spacing=0.12,
            horizontal_spacing=0.1
        )
        
        # Extract data for visualizations
        dates = list(self.analysis.keys())
        latest_date = dates[0] if dates else None
        
        if not latest_date:
            return fig
            
        latest_data = self.analysis[latest_date]
        
        # 1. Technical Indicators Radar Chart (converted to bar for simplicity)
        tech_indicators = latest_data['technical_indicators']
        if tech_indicators:
            # RSI and normalized values for comparison
            indicators = []
            values = []
            
            if 'rsi' in tech_indicators:
                indicators.append('RSI')
                values.append(tech_indicators['rsi'])
                
            if 'macd' in tech_indicators:
                indicators.append('MACD')
                values.append(tech_indicators['macd'] * 10)  # Scale for visibility
                
            if 'atr' in tech_indicators:
                indicators.append('ATR')
                values.append(tech_indicators['atr'] * 10)  # Scale for visibility
                
            if indicators:
                fig.add_trace(
                    go.Bar(x=indicators, y=values, name="Technical Indicators", 
                          marker_color=['#FF6B6B', '#4ECDC4', '#45B7D1']),
                    row=1, col=1
                )
        
        # 2. Investment Decisions Pie Chart
        decisions = latest_data['final_decisions']
        decision_counts = {}
        for decision in decisions.values():
            decision_counts[decision] = decision_counts.get(decision, 0) + 1
            
        if decision_counts:
            fig.add_trace(
                go.Pie(labels=list(decision_counts.keys()), 
                      values=list(decision_counts.values()),
                      hole=0.4,
                      marker_colors=['#FF6B6B', '#4ECDC4', '#45B7D1']),
                row=1, col=2
            )
        
        # 3. Sentiment Analysis
        sentiment_data = latest_data['sentiment_metrics']
        if sentiment_data:
            sentiment_labels = []
            sentiment_values = []
            
            if 'reddit_mentions' in sentiment_data:
                sentiment_labels.append('Reddit Mentions')
                sentiment_values.append(sentiment_data['reddit_mentions'])
                
            if 'sentiment_score' in sentiment_data:
                sentiment_labels.append('Sentiment Score')
                sentiment_values.append(sentiment_data['sentiment_score'])
                
            if sentiment_labels:
                fig.add_trace(
                    go.Bar(x=sentiment_labels, y=sentiment_values, 
                          name="Sentiment Metrics",
                          marker_color='#FFA726'),
                    row=2, col=1
                )
        
        # 4. Fundamental Metrics
        fundamental_data = latest_data['fundamental_metrics']
        if fundamental_data:
            fund_labels = []
            fund_values = []
            
            if 'revenue_growth' in fundamental_data:
                fund_labels.append('Revenue Growth %')
                fund_values.append(fundamental_data['revenue_growth'])
                
            if 'revenue_millions' in fundamental_data:
                fund_labels.append('Revenue (M)')
                fund_values.append(fundamental_data['revenue_millions'])
                
            if fund_labels:
                fig.add_trace(
                    go.Bar(x=fund_labels, y=fund_values, 
                          name="Fundamental Metrics",
                          marker_color='#66BB6A'),
                    row=2, col=2
                )
        
        # 5. Risk Assessment Gauge
        risk_data = latest_data['risk_assessment']
        risk_score = 50  # Default neutral
        
        if 'recommended_position_size' in risk_data:
            # Convert position size to risk score (higher position = higher confidence = lower risk)
            position_size = risk_data['recommended_position_size']
            risk_score = max(0, min(100, 100 - (position_size * 10)))
            
        fig.add_trace(
            go.Indicator(
                mode="gauge+number+delta",
                value=risk_score,
                title={'text': "Risk Level"},
                gauge={
                    'axis': {'range': [0, 100]},
                    'bar': {'color': "#FF6B6B"},
                    'steps': [
                        {'range': [0, 30], 'color': "#4ECDC4"},
                        {'range': [30, 70], 'color': "#FFA726"},
                        {'range': [70, 100], 'color': "#FF6B6B"}
                    ],
                    'threshold': {
                        'line': {'color': "red", 'width': 4},
                        'thickness': 0.75,
                        'value': 80
                    }
                }
            ),
            row=3, col=1
        )
        
        # 6. Price vs Moving Averages
        if 'closing_price' in tech_indicators:
            price_data = {
                'Current Price': tech_indicators.get('closing_price', 0),
                '50-SMA': tech_indicators.get('sma_50', 0),
                '200-SMA': tech_indicators.get('sma_200', 0),
                '10-EMA': tech_indicators.get('ema_10', 0)
            }
            
            # Filter out zero values
            price_data = {k: v for k, v in price_data.items() if v > 0}
            
            if price_data:
                fig.add_trace(
                    go.Scatter(x=list(price_data.keys()), 
                             y=list(price_data.values()),
                             mode='lines+markers',
                             name="Price vs MA",
                             line=dict(color='#9C27B0', width=3),
                             marker=dict(size=8)),
                    row=3, col=2
                )
        
        # 7. Decision Consensus
        all_decisions = list(decisions.values())
        buy_count = all_decisions.count('BUY')
        hold_count = all_decisions.count('HOLD')
        sell_count = all_decisions.count('SELL')
        
        fig.add_trace(
            go.Bar(x=['BUY', 'HOLD', 'SELL'], 
                  y=[buy_count, hold_count, sell_count],
                  name="Decision Consensus",
                  marker_color=['#4ECDC4', '#FFA726', '#FF6B6B']),
            row=4, col=1
        )
        
        # 8. Key Performance Metrics Table
        kpi_data = []
        
        # Compile KPI data
        if 'closing_price' in tech_indicators:
            kpi_data.append(['Current Price', f"${tech_indicators['closing_price']:.2f}"])
        if 'revenue_growth' in fundamental_data:
            kpi_data.append(['Revenue Growth', f"{fundamental_data['revenue_growth']}%"])
        if 'rule_of_40' in sentiment_data:
            kpi_data.append(['Rule of 40', f"{sentiment_data['rule_of_40']}%"])
        if 'rsi' in tech_indicators:
            kpi_data.append(['RSI', f"{tech_indicators['rsi']:.2f}"])
            
        if kpi_data:
            fig.add_trace(
                go.Table(
                    header=dict(values=['Metric', 'Value'],
                               fill_color='#E3F2FD',
                               align='left',
                               font=dict(size=12, color='#1976D2')),
                    cells=dict(values=[[row[0] for row in kpi_data], 
                                     [row[1] for row in kpi_data]],
                              fill_color='#F8F9FA',
                              align='left',
                              font=dict(size=11))
                ),
                row=4, col=2
            )
        
        # Update layout
        fig.update_layout(
            title=f"Investment Analysis Dashboard - {latest_data['company']} ({latest_date})",
            title_font_size=24,
            title_x=0.5,
            height=1200,
            showlegend=False,
            plot_bgcolor='white',
            paper_bgcolor='#F8F9FA',
            font=dict(family="Arial, sans-serif", size=10)
        )
        
        # Update all axes
        fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#E0E0E0')
        fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#E0E0E0')
        
        return fig
    
    def create_summary_report(self):
        """Create a summary report of the analysis"""
        report = {}
        
        for date, analysis in self.analysis.items():
            company = analysis['company']
            decisions = analysis['final_decisions']
            
            # Get most common decision
            decision_list = list(decisions.values())
            final_recommendation = max(set(decision_list), key=decision_list.count) if decision_list else "UNKNOWN"
            
            # Risk assessment
            risk_data = analysis['risk_assessment']
            position_size = risk_data.get('recommended_position_size', 'Not specified')
            stop_loss = risk_data.get('stop_loss_percent', 'Not specified')
            
            # Key metrics
            tech_indicators = analysis['technical_indicators']
            fundamental_metrics = analysis['fundamental_metrics']
            
            report[date] = {
                'company': company,
                'final_recommendation': final_recommendation,
                'current_price': tech_indicators.get('closing_price', 'N/A'),
                'rsi': tech_indicators.get('rsi', 'N/A'),
                'revenue_growth': fundamental_metrics.get('revenue_growth', 'N/A'),
                'position_size': position_size,
                'stop_loss': stop_loss
            }
            
        return report

In [3]:

# Example usage and demo
def demo_dashboard():
    """Demo function to show how to use the dashboard"""
    
    # Sample JSON data structure (you can replace this with actual data)
    sample_data = {
        "2025-08-01": {
            "company_of_interest": "PLTR",
            "trade_date": "2025-08-01",
            "market_report": "The closing price on August 1, 2025, is $158.35. 50-SMA: $139.85, 200-SMA: $96.60, 10-EMA: $155.04, MACD: 5.46, RSI: 58.63, ATR: 5.41 FINAL TRANSACTION PROPOSAL: BUY",
            "sentiment_report": "PLTR received 70 mentions on Reddit with a sentiment score of 58%. Rule of 40 score of 81%. Current price $160.66. FINAL TRANSACTION PROPOSAL: BUY",
            "fundamentals_report": "Q2 2025 Revenue: $1 billion, 48% year-over-year growth. Net Income: $327 million, 144% increase. Market Capitalization: $360 billion. FINAL TRANSACTION PROPOSAL: HOLD",
            "final_trade_decision": "BUY with 2.5% starter position, 12% stop-loss",
            "risk_debate_state": {
                "judge_decision": "BUY with 2.5% of portfolio at ~$160. Stop-loss at 12% below entry."
            },
            "news_report": "Global M&A activity shows strong growth. Palantir wins $10 billion U.S. Army contract showing positive momentum."
        }
    }
    
    # Create dashboard
    dashboard = InvestmentDashboard(json_data=sample_data)
    
    # Generate the dashboard
    fig = dashboard.create_dashboard()
    fig.show()
    
    # Get summary report
    summary = dashboard.create_summary_report()
    print("Summary Report:")
    print(json.dumps(summary, indent=2))
    
    return dashboard

In [4]:
# Run demo
demo_dashboard()

Summary Report:
{
  "2025-08-01": {
    "company": "PLTR",
    "final_recommendation": "BUY",
    "current_price": 1.0,
    "rsi": 58.63,
    "revenue_growth": 48,
    "position_size": 2.5,
    "stop_loss": 5
  }
}


In [ ]:
dashboard = InvestmentDashboard(json_file_path="path/to/your/file.json")
fig = dashboard.create_dashboard()
fig.show()
summary = dashboard.create_summary_report()


